In [1]:
import librosa
import pandas as pd
import torch.optim as optim
import numpy as np
import csv
from data_load import TextTransform
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
#from decoder import GreedyDecoder
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import librosa
from librosa.core import stft, magphase
from glob import glob
from torch import autograd
import csv
from data_load import CodeSwitchDataset
import zipfile

/home/dexter/Desktop/MSR challenge/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [2]:
def pad(wav, trans, lang):
    if lang == "Gujarati":
        max_len = 0
    elif lang == "Telugu":
        max_len = 529862
    elif lang == 'Tamil':
        max_len = 0
    else:
        raise Exception("Check Language")

    while len(wav) < max_len:
        diff = max_len - len(wav)
        ext = wav[:diff]
        wav = np.append(wav, wav[:diff])
        ratio = int(len(trans)*diff/len(wav))
        trans +=trans[:ratio]
    return wav, trans

def preprocess(data):
    #print(data)
    inputs = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    for (wav, sr, trans, lang) in data:
        wav, trans  = pad(wav, trans, lang)
        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        out = np.transpose(out, axes=(1, 0))

        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = torch.from_numpy(np.array([np.log(1 + x) for x in out]))
        #print(out.shape)
        inputs.append(out)
        labels.append(trans)
        input_lengths.append(out.shape[0])
        label_lengths.append(len(trans))
    inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    #spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    return inputs, labels, input_lengths, label_lengths

In [3]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=False):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [5]:
train_dataset = CodeSwitchDataset(lang = 'Telugu', mode = "train")
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


In [7]:
train_loader = DataLoader(train_dataset,
                          batch_size=16,
                          drop_last=True,
                          num_workers = 6,
                          sampler = train_sampler,
                         collate_fn = lambda x: preprocess(x))
test_loader = DataLoader(train_dataset,
                          batch_size=16,
                          drop_last=True,
                          num_workers = 6,
                          sampler = valid_sampler,
                         collate_fn = lambda x: preprocess(x))

In [8]:
device = torch.device('cuda')

In [9]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim = 29, num_layers = 4):
        super(Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        #self.drop = nn.Dropout(0.25)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, x):
        #print(type(self.lstm))
        #print(x.shape)
        lstm_out, hidden = self.lstm(x)
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.linear(lstm_out)
        return out, hidden

In [10]:
model = Model(input_dim=1025,
              hidden_dim=512,
              batch_size=16,
              output_dim=29,
              num_layers=4)
#model.half()
model = model.to(device)
criterion = nn.CTCLoss().to(device)
epochs = 40
optimizer = optim.Adam(model.parameters())

In [11]:
torch.cuda.empty_cache()

In [10]:
data_len = len(train_loader.dataset)
pbar = tqdm(enumerate(train_loader), total=len(train_loader))
for batch_idx, (_data) in pbar:
    #bi, wav, label = batch_idx, wav, label
    wav, labels, input_lengths, label_lengths = _data
    print(wav.shape)

  0%|          | 1/1061 [00:09<2:48:30,  9.54s/it]

torch.Size([16, 2409, 1025])


  0%|          | 2/1061 [00:10<2:02:44,  6.95s/it]

torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])


  1%|          | 6/1061 [00:11<1:26:32,  4.92s/it]

...

torch.Size([16, 2409, 1025])


 99%|█████████▉| 1051/1061 [17:43<00:19,  1.92s/it]

torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])


100%|██████████| 1061/1061 [17:48<00:00,  1.01s/it]

torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])
torch.Size([16, 2409, 1025])


labels

label_lengths

In [12]:
def train(model, device, train_loader, criterion, epoch, writer):
    model.train()
    data_len = len(train_loader.dataset)
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in pbar:
        #bi, wav, label = batch_idx, wav, label
        wav, labels, input_lengths, label_lengths = _data
        wav = wav.to(device)
        wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        #print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

        output, _ = model(wav)
        output = F.log_softmax(output, dim=1)
        
        output = output.transpose(0,1)
        #print(output.shape)
        
        loss = criterion(output, labels, input_lengths, label_lengths)
        #print(loss)
        loss.backward()
        
        optimizer.step()
        writer.add_scalar('Loss', loss, epoch*len(train_loader)+1)
        #writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join("checkpoints/", ckpt_model_filename)
        torch.save(model.state_dict(), ckpt_model_path)
        model.to(device).train()

In [13]:
def test(model, device, test_loader, criterion, epoch):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    with torch.no_grad():
        for batch_idx, _data in enumerate(test_loader):
            frr_far, eer = 0, 0
            inputs, labels, input_lengths, label_lengths = _data 
            inputs, labels = inputs.to(device), labels.to(device)

            output, _ = model(inputs)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=1)
            output = output.transpose(0, 1) # (time, batch, n_class)
            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            for j in range(len(decoded_preds)):
                if len(set(decoded_preds[j])) != len(set(decoded_targets[j])):
                    frr_far+=1
            eer = frr_far/len(decoded_preds)
            total_eer+=eer
            print("EER: ",eer)
            print("Total EER: ", total_eer)

In [14]:
for layer in model.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
writer = SummaryWriter('train_logs/')
for epoch in range(1, epochs+1):
    train(model, device, train_loader, criterion, epoch, writer)
    #test(model, device, test_loader, criterion, epoch, writer)
    
    
model.eval().cpu()
save_model_filename = "final_epoch_" + str(epoch + 1) + "_batch_id_" + str(batch_idx + 1) + ".model"
save_model_path = os.path.join("checkpoints", save_model_filename)
torch.save(model.state_dict(), save_model_path)

print("\nDone, trained model saved at", save_model_path)

  0%|          | 1/849 [00:07<1:51:27,  7.89s/it]

Train Epoch: 1 [0/16991 (0%)]	Loss: 68.732849


 12%|█▏        | 101/849 [01:52<10:15,  1.21it/s]

Train Epoch: 1 [1600/16991 (12%)]	Loss: 66.838272


 24%|██▎       | 201/849 [03:38<11:56,  1.11s/it]

Train Epoch: 1 [3200/16991 (24%)]	Loss: 66.381683


 35%|███▌      | 301/849 [05:23<09:49,  1.08s/it]

Train Epoch: 1 [4800/16991 (35%)]	Loss: 59.218315


 37%|███▋      | 316/849 [05:39<08:30,  1.04it/s]

RuntimeError: CUDA out of memory. Tried to allocate 1.97 GiB (GPU 0; 5.79 GiB total capacity; 1.27 GiB already allocated; 1.54 GiB free; 2.63 GiB reserved in total by PyTorch)

In [ ]:
len(trans)

In [ ]:
trans[0]

In [ ]:
wav[0].shape

In [ ]:
y_train = label
x_train = wav

In [ ]:
nb_train = len(x_train)
nb_features = len(x_train[0][0])

In [ ]:
nb_features

In [ ]:
nb_train

In [ ]:
x_train_len = np.asarray([len(x_train[i]) for i in range(nb_train)])
print(x_train_len)

In [ ]:
T = 189
n_class = 80
y = torch.tensor([[55, 43, 40, 62, 41, 44, 53, 40, 62, 41, 50, 53, 62, 58, 36, 54, 43, 44, 49, 42]])
output_length = torch.tensor(y.shape[1])

pred_model_idx = 79*torch.ones(T, dtype=torch.long)
pred_perf_idx = torch.cat([y[0], (n_class-1) * torch.ones(T-y.shape[1], dtype=torch.long)]) # the first idx are perfect with y, then padded with blanks
pred_model = torch.eye(n_class)[pred_model_idx].unsqueeze(1) # one-hot encoding
pred_perf = torch.eye(n_class)[pred_perf_idx].unsqueeze(1) # one-hot encoding

for input_length in [torch.tensor(y.shape[1]), torch.tensor(T)]:
    print("=============\ninput length:", input_length)
    print("perfect loss:", F.ctc_loss(F.log_softmax(pred_perf, dim=2), y, input_length, output_length, n_class-1, 'none', True))
    print("all_blank loss:", F.ctc_loss(F.log_softmax(pred_model, dim=2), y, input_length, output_length, n_class-1, 'none', True))


In [ ]:
wav, sr = librosa.load('Data/PartA_Telugu/Train/Audio/000010010.wav')
df = pd.read_csv('Data/PartA_Gujarati/Train/Transcription_LT_Sequence.tsv', header=None, sep='\t')

In [ ]:
pred_model.shape

In [ ]:
pred_perf

In [ ]:
qw = torch.rand((2,2,2))

In [ ]:
qw

In [ ]:
nn.Linear(2, 29)(qw).shape